# Data preprocessing script for fine-tuning GPT-3.5 or 4 model

## Downloading the Dataset from Kaggle

In [2]:
!pip install -q kaggle
from google.colab import files

# Choose the kaggle.json file that you downloaded
files.upload()

# Make directory named kaggle and copy kaggle.json file there.
!mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/

# Change the permissions of the file.
!chmod 600 ~/.kaggle/kaggle.json


# !kaggle datasets list

Saving kaggle.json to kaggle.json


In [3]:
# Downloading the dataset
!kaggle datasets download -d "kuldeepsingharya/english-to-hindi-parallel-dataset"

 92% 17.0M/18.5M [00:00<00:00, 68.6MB/s]
100% 18.5M/18.5M [00:00<00:00, 65.2MB/s]


In [4]:
!unzip "english-to-hindi-parallel-dataset.zip"

Archive:  english-to-hindi-parallel-dataset.zip
  inflating: newdata.csv             


## Importing the Dataset

In [5]:
import pandas as pd

dataset = pd.read_csv("newdata.csv")
# dataset.head()

In [6]:
dataset.head()

,Unnamed: 0,english_sentence,hindi_sentence
0,0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,2,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,3,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,4,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [7]:
dataset = dataset.iloc[:, 1:]
dataset

,english_sentence,hindi_sentence
0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
...,...,...
177601,He asked you saw tiger or not .,उसने पूछा टाइगर देखा या नहीं ।
177602,These words pricked like an arrow .,उसके यह शब्द तीर की तरह चुभ गए ।
177603,I slowly said no .,"मैंने धीरे से कहा , नहीं ।"
177604,There was no permission to take bike inside th...,पार्क में बाइक ले जाने की अनुमति नहीं थी ।


## Code for converting the Dataset into json format

At a high level, fine-tuning involves the following steps:

- Prepare and upload training data
- Train a new fine-tuned model
- Evaluate results and go back to step 1 if needed
- Use your fine-tuned model



**Example Format:**
```
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}
```



In [8]:
l = []
for val in dataset.iloc[:100, :].values:
  english = val[0]
  hindi = val[1]

  system = {"role":"system","content":"Marv is a translator that translate english language to hind language."}
  user = {"role":"user","content":english.strip()}
  assistant = {"role":"assistant","content":hindi.strip()}

  message = [system, user, assistant]
  l.append({"message": message})

In [9]:
print(l)

[{'message': [{'role': 'system', 'content': 'Marv is a translator that translate english language to hind language.'}, {'role': 'user', 'content': 'politicians do not have permission to do what needs to be done.'}, {'role': 'assistant', 'content': 'राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है .'}]}, {'message': [{'role': 'system', 'content': 'Marv is a translator that translate english language to hind language.'}, {'role': 'user', 'content': "I'd like to tell you about one such child,"}, {'role': 'assistant', 'content': 'मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,'}]}, {'message': [{'role': 'system', 'content': 'Marv is a translator that translate english language to hind language.'}, {'role': 'user', 'content': 'This percentage is even greater than the percentage in India.'}, {'role': 'assistant', 'content': 'यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।'}]}, {'message': [{'role': 'system', 'content': 'Marv is a translator that translate english langua

## Saving the conveted json data into jsonl file

In [10]:
import json

with open("data.jsonl", mode='w', encoding='utf-8') as fp:

  for item in l:
    fp.write(json.dumps(item, ensure_ascii=False) + '\n')